In [1]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation
import numpy as np
import pandas as pd


Using TensorFlow backend.


In [2]:
inputs = pd.read_csv('traincleaned.csv')
inputs = inputs.drop(columns=['SalePrice', 'Id'])
# inputs = inputs[['LotArea', 'MSSubClass']]
inputs = inputs.fillna(0)
inputs = pd.get_dummies(inputs)
inputs.head()
inputs = inputs.values
inputs

array([[0.000e+00, 6.000e+01, 6.500e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.000e+00, 2.000e+01, 8.000e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+00, 6.000e+01, 6.800e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.457e+03, 7.000e+01, 6.600e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.458e+03, 2.000e+01, 6.800e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.459e+03, 2.000e+01, 7.500e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00]])

In [3]:
prices = pd.read_csv('train.csv')
prices = prices['SalePrice']
prices = np.array(prices)
prices

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [4]:
print(inputs.shape, prices.shape)

(1460, 305) (1460,)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X_train, X_test, y_train, y_test = train_test_split(inputs, prices, random_state=1)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
model = Sequential()
model.add(Dense(units=1800,
                activation='relu',
               input_dim=inputs.shape[1]))
model.add(Dropout(0.75))
model.add(Dense(units=1800,
                activation='relu',
               input_dim=inputs.shape[1]))
model.add(Dropout(0.75))
model.add(Dense(units=1,
               activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1800)              550800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1800)              3241800   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1801      
Total params: 3,794,401
Trainable params: 3,794,401
Non-trainable params: 0
___________

In [7]:
for x in range(10):
    model.fit(
        X_train_scaled,
        y_train,
        epochs=1,
        shuffle=True,
        verbose=0
    )
    if x % 1 == 0:
            test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
            train_loss, train_accuracy = model.evaluate(X_train_scaled, y_train, verbose=2)
            print(x, round(train_accuracy, 4), round(test_accuracy,4), abs(round(train_accuracy - test_accuracy,4)))
            
    if abs(round(train_accuracy - test_accuracy,4)) < 500:
        break

Instructions for updating:
Use tf.cast instead.
0 181202.2179 175647.282 5554.9359
1 171147.7524 166003.0066 5144.7458
2 130711.8542 127138.876 3572.9783
3 57770.4436 54559.9393 3210.5044
4 46576.5521 44435.6385 2140.9136
5 40342.5595 39172.3342 1170.2253
6 36578.1376 36614.4375 36.3


In [8]:
error = 0

for x in range(len(X_test_scaled)/2):
    error += abs((model.predict(X_test_scaled)[x][0] - y_test[x])/y_test[x])
    
print('Average Error:', error/x)

TypeError: 'float' object cannot be interpreted as an integer